In [13]:
import requests
from bs4 import BeautifulSoup
import time

In [14]:
class Content:
    def __init__(self, topic, url, title, body):
        self.topic = topic
        self.url = url
        self.title = title
        self.body = body

    def print(self):
        print('New articel found for toplic: {}'.format(self.topic))
        print('URL: {}'.format(self.url))
        print('TITLE: {}'.format(self.title))
        print("BODY: \n {}".format(self.body))

In [15]:
class Website:
    def __init__(self, name, url, searchUrl, resultListing, resultUrl, absoluteUrl, titleTag, bodyTag):
        self.name = name
        self.url = url
        self.searchUrl = searchUrl
        self.resultListing = resultListing
        self.resultUrl = resultUrl
        self.absoluteUrl = absoluteUrl
        self.titleTag = titleTag
        self.bodyTag = bodyTag

In [18]:
class Crawler:
    def getPage(self, url):
        try:
            req = requests.get(url)
        except requests.exceptions.RequestException:
            return None
        return BeautifulSoup(req.text, 'html.parser')
    
    def safeGet(self, pageObj, selector):
        childObj = pageObj.select(selector)
        if childObj is not None and len(childObj) > 0:
            return childObj[0].get_text()
        else :
            return ' '
            
    def getAllBody(self, pageObj, selector):
        childObj = pageObj.select(selector)
        bodyText = ""
        if childObj is not None:
            for i in range (len(childObj)):
                bodyText = bodyText + childObj[i].get_text() + '\n'
            return bodyText
        else:
            return ''
    
    def search(self, topic, site):
        print('searchUrl+topic:', site.searchUrl + topic)

        bs = self.getPage(site.searchUrl + topic)
        searchResults = bs.select(site.resultListing)

        for result in searchResults:
            url = result.select(site.resultUrl)[0].attrs['href']
            if(site.absoluteUrl):
                bs = self.getPage(url)
            else :
                bs = self.getPage(site.url + url)
            if bs is None :
                print('Somthing was wrong with tha page or URL. Skipping')
                return
            title = self.safeGet(bs, site.titleTag)
            body = self.getAllBody(bs, site.bodyTag)

            if title != '' and body != '':
                content = Content(topic, url, title, body)
                content.print()

In [19]:
crawler = Crawler()
siteData1 = [
    ['Reuters', # Website.name
    'http://reuters.com', # Website.url
    'http://www.reuters.com/search/news?blob=', # Website.searchUrl: 검색을 위한 URL
    'div.search-result-content', # Website.resultListing: 검색 결과에 대한 정보
    'h3.search-result-title > a', # Website.resultUrl: 결과에서 URL을 추출할 때 사용할 태그
    False, # Website.absoluteUrl 사용 여부
    'h1', # Website.titleTag
    'p.Paragraph-paragraph-2Bgue'] # Website.bodyTag
]

sites = []
for row in siteData1:
    sites.append(Website(row[0], row[1], row[2], row[3], row[4], row[5], row[6], row[7]))

topics = ['python']
for topic in topics:
    print('GETTING INFO ABOUT: ' + topic)
    for targetSite in sites:
        crawler.search(topic, targetSite)

GETTING INFO ABOUT: python
searchUrl+topic: http://www.reuters.com/search/news?blob=python
New articel found for toplic: python
URL: /article/idUSKCN11S04G
TITLE: Python in India demonstrates huge appetite
BODY: 
 A 20 feet rock python was caught on camera in Junagadh district of India’s western Gujarat state with a swollen stomach after it consumed an antelope on Tuesday (September 20).
Residents informed authorities at Girnar Wildlife Sanctuary after they spotted the reptile lying in discomfort in a field.
In view of the massive swelling of the python’s stomach, the forest authorities suspect that it gobbled up a full-grown ‘nilgai’ or blue bull.
The python - unable to move now - was rescued by the forest personnel and has been put under observation.
“We will keep it (python) under observation. We will release it back in the wild once it digests the antelope and the swelling subsides,” said Assistant Conservator of Forest, S.D. Tilala.
A blue bull is far larger than an ideal prey for